<a href="https://colab.research.google.com/github/easyhardhoon/machine_learning/blob/master/ML9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

활성화 함수로 이루어진 신경망 계층들.

계층이란 노드(동그라미)를 의미함. 

예를 들어 relu 계층은  >-- x --> RELU >-- y --> 이런식임

추가 : 계층은 최대한 단순화된 노드의 집합이라고 이해해도 될듯

relu는 계산이 더 이상 나누어지지 않아 relu가 하나의 노드처럼 인식

sigmoid는 계산이 여러개로 세분화됨. 그 여러개가 각각의 노드이고

sigmoid 계층은 이러한 노드들의 집합

#결국은 sigmoid도 +, * , RELU 처럼 하나의 노드처럼 정리 가능  

# 2. sigmoid 계층

sigmoid함수 y = 1/1+exp(-x)

세분화된 노드들로 순전파와 역전파를 구해보면, 

역전파의 결과물이 순전파의 입력 x와 출력 y만으로 계산할 수 있다

또 그 식을 정리하면 역전파에서 dL/dy가 들어오면 dL/dy * y(1-y) 로 정리가능

즉 위 relu처럼 하나의 sigmoid 노드로 정리할 수 있다. 





In [1]:
import numpy as np

class Sigmoid:
  def __init__(self):
    self.out = None #아무튼간에 순전파떄의 출력 y가 역전파떄 쓰이기 떄문에 y를 기록해놔야함
  def forward(self,x):
    out = 1 / (1 + np.exp(-x))
    self.out = out 
    return out
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx
  

#신경망 순전파의 흐름

▶ 1. Y = np.dot(X,W) + B

▶ 2. Y = 활성화함수(Y). 

▶ 3. 다음 층으로 전파

여기서 (1)의 행렬곱을 어파인 변환이라고 함. 어파인 변환을 수행하는 처리를

(1)의 과정을 "Affine 계층" 이라고 부른다

# 3. Affine 계층

변수가 행렬(다차원배열)이긴 하지만 행렬의 원소마다 전개해보면 스칼라값을 이용했던 지금까지의 계산 그래프와 같은 순서이다.

핵심은 : 변수가 행렬이기 때문에 , 행렬에 대응하는 차원의 원소 수가 일치하도록 곱을 조립하는 것이다 (특히 행렬곱)



#Affine 계산 그래프에서

X와 dL/dx는 같은 형상임. 순방/역전파의 개념 이해. 

마찬가지로 W와 dL/dw도 같은 형상

W_t의 t는 전치행렬을 의미. 전치행렬은 W의 (i,j) 를 (j,i)로 바꿔줌

**행렬의 곱에서 행렬의 형상을 맞춰줘야 하는 원리에 의해**,

W_t 같은 전치행렬이 도입된다는 점을 이해

ex) dL/dY 도트 (???) = dL/dX ,  ???에 W가 와야하는 상황

-->  (3, ) 도트 (???) = ( 2,)  . 근데 W의 형상은 (2,3)이다

#기본적으로 (3,)을 (1,3)으로 이해하자. 

???에는 (3,2)가 와야함을 알 수 있다 

그래서 W의 형상을 뒤집은 W**T, 즉 전치행령이 도입




#입력값 딜레마

데이터가 1개든 여러개든 28*28크기로 인해 행렬의 느낌으로 "배열" 형식이 들어가서, 헷갈릴 수 있다.

위에서 나오듯, "데이터 1개로 된 신경망" 과   "데이터 n개, 즉 배치용 신경망"이 따로 있다고 생각하면 편할 듯하다. 

즉 데이터가 하나가 들어가든 배치형식으로 한꺼번에 여러개가 들어가든
입력형식은 행렬이다.

하나이면 1차원 행렬, 배치이면 2차원 행렬 

자세한건 잠시 뒤에 다시 나올듯

#배치용 Affine 계층

지금까지의 affine 계층은 입력 데이터로 X하나만을 고려한 것이었다.

그런데 데이터를 N개 묶어 순전파하는 경우, 즉 배치용 Affine 계층을 생각해본다.


배치용 Affine 계층과 입력 데이터 1개 Affine 계층의 차이

--> 입력인 X의 형상이 (2,) 에서 (N,2)로 되었다. 

--> 입력 데이터 1개이나 배치용이나 기본적으로 배열(행렬)형식인건 맞다.

(지금까지의 계산 그래프에는 노드 사이에 스칼라값이 들어갔었음)

배치용이면 데이터의 차원이 추가되어 2차원 배열이 되는 것 뿐이다

#배치용 Affine 계층에서의 편향에 대하여

Y = np.dot(X,W) + B ------> 이 과정이 Affine 과정입니다.

 여기서 "+" 노드에 주목하자


순전파의 편향 덧셈은 각각의 데이터에 더해진다. 

( ex) 2차원 배열에 1차원 배열이 더해지는 꼴. numpy의 브로드캐스트 기능 덕분에 구현이 가능하다 )

**그래서 역전파 때는 각 데이터의 역전파 값이 편향의 원소에 "모여야" 한다**

---> "+"노드는 역전파때 입력값을 그대로 출력한다. 

--->그런데 입력값이 2차원인데 출력(편향)은 1차원이니까, 이 2차원값을 1차원 형식으로 바꿔준다는 뜻이다

---> 2차원 원소들을 1차원으로 "모은다"의 느낌으로 이해하자

In [4]:
dY = np.array([[1,2,3], [4,5,6]])
dB = np.sum(dY, axis=0) # + 노드의 역전파이니 입력값을 그대로 출력해야하는데 차원이 2차원에서 1차원으로 줄여야함
#두 데이터에 대한 미분 (입력값)을 데이터마다 더해서 구하는 느낌. 
#그래서 np.sum에서 0번쨰 축(데이터 단위)에 대해 총합을 구했음.
print(dY)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [5]:
class Affine:
  def __init__(self,W,b):
    self.W = W #역전파떄 필요
    self.b = b 
    self.x = None #역전파때 필요
    self.dw = None 
    self.db = None 
    # 나머지들은 안쓰이는데 왜 따로 저장하지..?
    # 나중에 쓰일듯!
  def forward(self, x):
    #x는 입력데이터
    self.x = x
    out = np.dot(x,self.W) + self.b
    return out
  def backward(self,dout):
    #최종 갈래는 3갈래. dx, dw, db
    #아래 이상한 코드는 dot 노드의 역전파의 공식입니다. 
    dx  = np.dot(dout, self.W.T) #이유 : +역전파는 입력값 그대로고 도트 역전파는 dot&전치행렬의 개념이기 떄문
    self.dW = np.dot(self.x.T, dout) #도트 역전파의 공식입니다. 순전파 입력값의 전치행렬에 dout의 도트를 한게 공식입니다.
    self.db = np.sum(dout, axis=0)  #앞서 말한 이유입니다.
    #기본적으로 batch가 아닌 데이터가 하나만 들어올떄도 코드가 
    #동일하게 실행되도록 설계. 
    return dx
    